# CS4035 - Cyber Data Analytics
## Lab 1 - Fraud data

### Submit on brightspace (zip file with the name Group_xx.zip)
(i) This jupyter file completed with code, plots, figures and report for each question. Additional plots and figures can be created for explanation before the end of each main question. Lab 1 contains 5 main questions, including the bonus. Write the code or explanation below each sub question. For the explanations, include what you would normally include in the report for this lab assignment, for example data pre-processing, hypothesis tested, approach, results, etc.
(ii) The libraries needed to run this file. Except for numpy, scikit-learn, pandas, matplotlib

Your peers should be able to use the readme section for instructions and be able to run this file. 

Make sure to keep your answers concise. Maximum number of words is 1000, which you can count with the code below. (You can add around 600 words since we start at around 400).

In [14]:
# If this cell does not work try running `pip install nbformat`

import io
from IPython import nbformat

nb = nbformat.read("Lab1-2021.ipynb", nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))

print("Word count:", word_count)

Word count: 863


## Group Number :

## Student 1 
### Name : Martijn van Meerten
### ID : 4387902

## Student 2
### Name : Yannick Haveman
### ID : 4299078

## Readme - Provide instructions - libraries used, location of the data file, etc. Keep it short. Remember your peers will not debug your code and should be able to reproduce the exact output you provide.

In [2]:
# Absolute path to the credit card fraud data
data_path = "C:\\Users\Yannick\Cyber Data Analytics\data_for_student_case.csv"

## 1. Visualization task – 1 A4

### 1a. Plot visulations 

### 1b. Describe the visualizations and provide relavant explanations of features and relations.

## 2. Imbalance task – 1 A4 – Individual

## SMOTE

### 2a. Print ROC curves

### 2b. Analyse the performance of the classifiers. Explain which method performs best.

### 2c. Is using SMOTE a good idea? Why (not)?

## 3. Privacy task – 1 A4 – Individual

### Preprocessing

In [3]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import re

# Read data
df1 = pd.read_csv(data_path)

# Class label 
df1 = df1.loc[~(df1['simple_journal'] == 'Refused')]
df1.loc[df1['simple_journal'] == 'Chargeback', 'simple_journal'] = 1 #fraud
df1.loc[df1['simple_journal'] == 'Settled', 'simple_journal'] = 0

# Various feature label encoding
card_enc = LabelEncoder()
card_enc.fit(df1['card_id'])
df1['card_id'] = card_enc.transform(df1.card_id)

ip_enc = LabelEncoder()
ip_enc.fit(df1['ip_id'])
df1['ip_id'] = ip_enc.transform(df1.ip_id)

# 4. Date
df1['creationdate'] = pd.to_datetime(df1['creationdate'])
df1['date'] = df1['creationdate'].dt.date

#Encoding, Cleaning the data
df1.loc[df1['cardverificationcodesupplied'].isna(),'cardverificationcodesupplied'] = False
df1.loc[df1['issuercountrycode'].isna(),'issuercountrycode'] = 'ZZ'
df1.loc[df1['shoppercountrycode'].isna(),'shoppercountrycode'] = 'ZZ'

unique_issuer_cc = df1['issuercountrycode'].unique()
unique_shopper_cc = df1['shoppercountrycode'].unique()
both = np.append(unique_issuer_cc, unique_shopper_cc)
df_countrycodes = pd.DataFrame(both)
unique_codes = df_countrycodes[0].unique()
enc = LabelEncoder()
enc.fit(unique_codes)
df1['issuercountrycode'] = enc.transform(df1.issuercountrycode)
df1['shoppercountrycode'] = enc.transform(df1.shoppercountrycode)
def conv(row):
    currency_dict = {"BGN": 1.9558, "NZD": 1.6805, "ILS": 4.0448, "RUB": 72.2099, "CAD": 1.5075, "USD": 1.1218,
                     "PHP": 58.125, "CHF": 1.1437, "ZAR": 16.0224, "AUD": 1.5911, "JPY": 124.93, "TRY": 6.6913,
                     "HKD": 8.8007, "MYR": 4.6314, "THB": 35.802, "HRK": 7.413, "NOK": 9.6678, "IDR": 15953.68,
                     "DKK": 7.4646, "CZK": 25.659, "HUF": 322.97, "GBP": 0.86248, "MXN": 21.2829, "KRW": 1308.01,
                     "ISK": 136.2, "SGD": 1.5263, "BRL": 4.405, "PLN": 4.2868, "INR": 78.0615, "RON": 4.7596,
                     "CNY": 7.5541, "SEK": 10.635}
    return row['amount'] / (currency_dict[row['currencycode']]*100)

df1['amount_eur'] = df1.apply(lambda x: conv(x), axis=1)


enc1 = LabelEncoder()
enc1.fit(df1['txvariantcode'])
df1['txvariantcode'] = enc1.transform(df1.txvariantcode)

enc2 = LabelEncoder()
enc2.fit(df1['currencycode'])
df1['currencycode'] = enc2.transform(df1.currencycode)

enc3 = LabelEncoder()
enc3.fit(df1['shopperinteraction'])
df1['shopperinteraction'] = enc3.transform(df1.shopperinteraction)

df1['accountcode'] = df1['accountcode'].apply(lambda x: re.sub('Account','',x))
df1['accountcode_cc'] = 0
df1.loc[(df1['accountcode'] == 'UK'),'accountcode_cc'] = 'GB'
df1.loc[(df1['accountcode'] == 'Mexico'),'accountcode_cc'] = 'MX'
df1.loc[(df1['accountcode'] == 'Sweden'),'accountcode_cc'] = 'SE'
df1.loc[(df1['accountcode'] == 'APAC'),'accountcode_cc'] = 'APAC'

enc4 = LabelEncoder()
enc4.fit(df1['accountcode'])
df1['accountcode'] = enc4.transform(df1.accountcode)

enccc = LabelEncoder()
enccc.fit(df1['accountcode_cc'])
df1['accountcode_cc'] = enccc.transform(df1.accountcode_cc)

enc5 = LabelEncoder()
enc5.fit(df1['cardverificationcodesupplied'])
df1['cardverificationcodesupplied'] = enc5.transform(df1.cardverificationcodesupplied)

df1.loc[df1['mail_id'].str.contains('na',case=False),'mail_id'] = 'email99999'

enc6 = LabelEncoder()
enc6.fit(df1['mail_id'])
df1['mail_id'] = enc6.transform(df1.mail_id)

df1['bookingdate'] = pd.to_datetime(df1['bookingdate'])

df1.loc[df1['cvcresponsecode'] > 2,'cvcresponsecode'] = 3

#Feature Engineering
df1['countries_equal'] = (df1['shoppercountrycode'] == df1['issuercountrycode'])
df1.loc[df1['countries_equal'] == False,'countries_equal'] = 0
df1.loc[df1['countries_equal'] == True,'countries_equal'] = 1

df1['day_of_week'] = df1['creationdate'].dt.dayofweek

df1['hour'] = df1['creationdate'].dt.hour

original_data_df = df1.copy()

keep_cols = [
    "issuercountrycode",
    "txvariantcode",
    "bin",
    "amount",
    "currencycode",
    "shoppercountrycode",
    "shopperinteraction",
    "cardverificationcodesupplied",
    "cvcresponsecode",
    "accountcode",
    "mail_id",
    "ip_id",
    "card_id",
    "accountcode_cc",
    "simple_journal",
    "day_of_week",
    "hour"
]

df1 = df1[keep_cols]

y = df1["simple_journal"].to_numpy().astype(int)
df1.drop(columns="simple_journal", inplace=True)
df2 = df1

### 3a. Print ROC curves

In [ ]:
classifiers = [RandomForestClassifier(random_state=1), GaussianNB(), DecisionTreeClassifier(random_state=1)]
classifier_names = ['Random forest', 'GaussianNB', 'Decision Tree']
ranking_names = ['rank_by_seller_first', 'rank_by_seller_last', 'rank_by_day_of_week']

colors = ['blue', 'red', 'green']
fig, axs = plt.subplots(1, 3, figsize=(17,5))
k_fold = StratifiedKFold(n_splits=10)
for j in range(len(ranking_names)):
    if j == 0:
        df1 = df2
        df1['rank_transaction_by_seller_first'] = df1.groupby(['accountcode', 'shoppercountrycode'])['cvcresponsecode'].rank(method='first')
        df1.sort_values(by=['accountcode', 'rank_transaction_by_seller_first'])

        df1[df1['rank_transaction_by_seller_first'] == 1]

        X = df1.to_numpy().astype(float)
    elif j == 1:
        df1 = df2
        df1['rank_transaction_by_seller_last'] = df1.groupby(['accountcode', 'shoppercountrycode'])['cvcresponsecode'].rank(method='last')
        df1.sort_values(by=['accountcode', 'rank_transaction_by_seller_last'])

        df1[df1['rank_transaction_by_seller_last'] == 1]

        X = df1.to_numpy().astype(float)        
    else:
        df1 = df2
        df1['rank_transaction_by_day_of_week'] = df1.groupby(['accountcode', 'shoppercountrycode'])['day_of_week'].rank(method='first')
        df1.sort_values(by=['accountcode', 'rank_transaction_by_day_of_week'])

        df1[df1['rank_transaction_by_day_of_week'] == 1]

        X = df1.to_numpy().astype(float)
    for i in range(len(classifier_names)):
        start_time = time.time()
        
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        
        for train_index, test_index in k_fold.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            classifiers[i].fit(X_train, y_train)

            y_pred_proba = classifiers[i].predict_proba(X_test)[:, 1]
            fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
            roc_auc = auc(fpr, tpr)
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(roc_auc)
        
        print(classifiers[i], "--- %s seconds ---" % (time.time() - start_time))

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)

        axs[j].plot(mean_fpr, mean_tpr, label=r'%s (AUC = %0.2f $\pm$ %0.2f)' % (classifier_names[i], mean_auc, std_auc), color=colors[i])

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        axs[j].fill_between(mean_fpr, tprs_lower, tprs_upper, color=colors[i], alpha=.2)
    
    axs[j].legend(loc="lower right")
    axs[j].set(xlim=[0, 1], ylim=[0, 1], title=ranking_names[j], xlabel="False positive rate", ylabel="True positive rate")
    axs[j].plot([0, 1], [0, 1], c="gray", linestyle="--")

fig.suptitle('Results after Maxing')

### 3b. Analyse the performance of the classifiers. Explain which method performs best.

Looking at all of the classifiers used to test the data provided it seems that the Random Forest classifier outperforms the other classifiers used. The classifier does provide really good results, however, but the time it takes to run is much higher than the other methods. The Gaussian classifer scores a bit lower than the Random Forest, but the time it takes to calculate is less than 1/100. Both the running time and performance of the Decission Tree classifier seem to be alright, but it's outperformed by the Gaussian Classifier. Hence, it can be determined that the Random Forest classifier is the best to use when one has the time to perform the calculations, but if one wants to arrive at results quickly it's best to use the Gaussian Classifier

### 3c. Can you explain the performance difference for the different classifiers? Is it advisable to protect people’s privacy using rank-swapping? Why (not)?

Looking at all of the classifiers used to test the data provided it seems that the Random Forest classifier outperforms the other classifiers used. The classifier does provide really good results, however, but the time it takes to run is much higher than the other methods. The Gaussian classifier scores a bit lower than the Random Forest, but the time it takes to calculate is less than 1/100. Both the running time and performance of the Decision Tree classifier seem to be alright, but it's outperformed by the Gaussian Classifier. Hence, it can be determined that the Random Forest classifier is the best to use when one has the time to perform the calculations, but if one wants to arrive at results quickly it's best to use the Gaussian Classifier

The Gaussian classifier is very easy to run calculation wise, which is a very important deciding factor for the run time. Nonetheless, the speed of the calculations aren't that detrimental for the results as it's still very accurate. The decision Tree is known for its simplicity and ability to still provide correct results regardless. It's good for creating accurate models, but as seen it is outperformed by other classifiers in this situation. However, I expect it to perform better in other scenarios such as data mining. The Random Forest is built upon the Decision Tree classifier of which the Forest uses an average of Trees in the forest. All of these Trees contain a subset of features.

I'd say that it is advisable to use Rank-Swapping as it performs well and quickly. Given that the privacy is also harnessed adds to the strength of the method. The ranking however, is important for the final result and that should be taken into when deciding on how to use the method.

## 4. Classification task – 2 A4

### Preprocessing

### Classification

### 4a. Print relevant plots and metrics with clear headings.

### 4b. Explain the applied data pre-processing steps, learning algorithms, and post-processing steps or ensemble methods. Compare the performance of the two algorithms, focusing on performance criteria that are relevant in practice, use 10-fold cross-validation.

## 5. Bonus task – 1 A4

### 5a. Provide code and report below